In [1]:
#!pip install python-numexpr
#!pip install python-pip build-essentials
#!pip install bottleneck

import pandas as pd
!pip install pandasql
from pandasql import sqldf

# Import Data Source

In [ ]:
#import dataset all

df_src2 = pd.read_excel('DMSYear_All.xlsx')

In [3]:
df_src = df_src2.copy()

In [4]:
# df_src.isna().sum()
df_src = df_src.dropna(axis=0, subset=['Date'])
df_src[df_src.columns[0]].count()

101029

In [5]:
# convert 2 column to one

df_src['Datetime'] = pd.to_datetime(df_src.Date.astype(str) + ' ' + df_src.Time.astype(str))
df_src['Datetime'] = df_src['Datetime'].astype('object')

In [6]:
# rename column

df_src= df_src.rename(columns={"Vehicle":"Plate_Number", "Operator":"Nama_Driver", "Time":"Time2", "Datetime":"Time",
                               "Shift":"shift", "Category":"Warning_Type", "Speed (Km/h)":"Speed", "SID/Code":"Kode_Drivers",
                               "Operator":"Nama_Driver", "Latitude":"lat", "Longitude":"lng", "Media":"media", 
                               "Area":"Nama_Area", "Heading":"dir"})

In [7]:
# cleaning data

df_src['shift'] = df_src['shift'].apply(pd.to_numeric).astype('Int64')
df_src = df_src.dropna(axis=0, subset=['Kode_Drivers'])

df_src[df_src.columns[0]].count()

67576

In [ ]:
# cleaning duplicate data

df_src.duplicated(subset=['Plate_Number', 'Time', 'Kode_Drivers', 'Warning_Type'], keep='first')
# df_src.duplicated(subset=['PK1'])

In [ ]:
pd.set_option('display.max_columns', None)
# df_final.head()
df_src.tail()

In [9]:
# count total row

# df_src = df_src.drop_duplicates(subset=['Time', 'Kode_Drivers', 'Warning_Type'], keep='first')
df_src[df_src.columns[0]].count()

67576

In [ ]:
# see missing value

df_src.isnull().sum()

# Read DB Destination

In [12]:
import mysql.connector
import pandas as pd


# read db destination

mydb = mysql.connector.connect(
    host= '#YOURHOST',
    port= #YOURPORT,
    user= '#YOURUSER',
    password= '#YOURPASSWORD',
    database="#YOURDATABASE""
)
if mydb.is_connected():
    print("Connected")

Connected


In [ ]:
# Open data destination MySQL

mycursor_dst = mydb.cursor()
data_dst = pd.read_sql_query("select * from dms2.DMS_Backup", con=mydb)

In [25]:
# delete missing value

data_dst = data_dst.dropna(axis=0, subset=['Kode_Drivers'])

In [ ]:
# see missing value

data_dst.isnull().sum()

In [27]:
# count total row

data_dst[data_dst.columns[0]].count()

73284

# Append Data source to Database Destination

In [28]:
# Select data

df1 = pd.DataFrame(df_src, columns= ['Plate_Number', 'Time', 'shift', 'Warning_Type', 'Speed', 'Nama_Area', 'Kode_Drivers', 
                                  'Nama_Driver', 'media'])
df2 = pd.DataFrame(data_dst, columns= ['Plate_Number', 'Time', 'shift', 'Warning_Type', 'Speed', 'Nama_Area', 'Kode_Drivers', 
                                  'Nama_Driver', 'media'])

df_final = pd.concat([df1, df2], ignore_index=True)
df_final[df_final.columns[0]].count()

140860

In [29]:
# Drop duplicate

df_final = df_final.drop_duplicates(subset=['Plate_Number', 'Time', 'Kode_Drivers', 'Warning_Type'], keep='first')
# df = df.drop_duplicates()

df_final[df_final.columns[0]].count()

77613

In [ ]:
# read all column

pd.set_option('display.max_columns', None)
# df_final.head()
df_final

In [ ]:
# cleaning data

# column = ['ID', 'acc', 'id_warning_type', 'flag_alarm']
# for i in column:
#     df[i] = df[i].astype('object')

df_src['shift'] = df_src['shift'].astype('object')
df_src = df_src.astype(object).where(pd.notnull(df_src), None)
df_src.info()

In [33]:
# delete null value

df_final = df_final.fillna(0)

In [ ]:
# see missing value 

df_final.isnull().sum()

In [ ]:
# DANGERR!!!!!
# insert data to mySQL Database

mycursor= db_dst.cursor()

sql = "INSERT INTO dms_backup.db_all_mitra_all(Plate_Number, Time, shift, Warning_Type, Speed, Nama_Area, Kode_Drivers, Nama_Driver, media) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)"

for i in range(0,len(df_src)):
    val=((df_final['Plate_Number'][i]),(df_final['Time'][i]),(df_final['shift'][i]),(df_final['Warning_Type'][i]),(df_final['Speed'][i]),(df_final['Nama_Area'][i]),(df_final['Kode_Drivers'][i]),(df_final['Nama_Driver'][i]),(df_final['media'][i]))
    #(val)
    mycursor.execute(sql,val)
    
    db_dst.commit()

print(mycursor.rowcount, "was inserted.")